In [1]:
import pandas as pd
import datetime as dt
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns
import japanize_matplotlib
from janome.tokenizer import Tokenizer
from janome.analyzer import Analyzer
from janome.charfilter import *
from janome.tokenfilter import *
from wordcloud import WordCloud
import collections
import pickle
sns.set(font="IPAexGothic")

# 現在の最大表示列数の出力
pd.get_option("display.max_columns")
# 最大表示列数の指定（ここでは50列を指定）
pd.set_option('display.max_columns', 50)

# 現在の最大表示列数の出力
pd.get_option("display.max_rows")
# 最大表示列数の指定（ここでは50列を指定）
pd.set_option('display.max_rows', 100)

sns.set(font=['IPAexGothic'])
%matplotlib inline

/Users/kanekotakafumi/anaconda3/lib/python3.7/site-packages/japanize_matplotlib/__init__.py:13: MatplotlibDeprecationWarning: 
The createFontList function was deprecated in Matplotlib 3.2 and will be removed two minor releases later. Use FontManager.addfont instead.
  font_list = font_manager.createFontList(font_files)


In [2]:
dataset_path = '/Users/kanekotakafumi/github/fake_news_detection_research/data/dataset/'
dict_ = pickle.load(open(dataset_path+'datset_dict.pkl','rb'))

In [3]:
def word_segmentation(text:str, replace_dict:dict, exclude_task_list:list, a):
    
    token_list = []
    for token in a.analyze(text):
        token = token.translate(str.maketrans(replace_dict)) # 邪魔な文字を除く
        if token not in exclude_task_list and not token.isdecimal(): # 意味ないワードを除く
            token_list.append(token)
            token_list = [s for s in token_list if not s.startswith('#')]
            
    token_str = ' '.join(token_list)
    
    return token_str

In [5]:
replace_dict = {'[':'', ']':'', '/':'', '+':'', '(':'', ')':'', '等':'', ',':'', '.':'', '<':'', '>':'', '-':'', '?':'', ':':'', '|':''}
exclude_task_list = ['new', 'rt', '籏智広太', 'インファクト', 'ファクトチェック', 'factcheck', 'infact', 'こび', 'ナビ']

# analyzerモジュールで形態素分析
char_filters = [UnicodeNormalizeCharFilter()]

tokenizer = Tokenizer()

token_filters = [CompoundNounFilter(),
                 POSStopFilter(['記号']),
                 LowerCaseFilter(),
                 ExtractAttributeFilter('surface')]

a = Analyzer(char_filters=char_filters, tokenizer=tokenizer ,token_filters=token_filters)

for key in dict_.keys():
    text_word_segmentation_list = []
    for text in dict_[key]['text']:
        text_word_segmentation_list.append(word_segmentation(text, replace_dict, exclude_task_list, a))
    dict_[key]['text_word_segmentation'] = text_word_segmentation_list

In [20]:
"""
Word2vec model.
"""
import smart_open
smart_open.open = smart_open.smart_open
import argparse
import logging
from pprint import pprint

from gensim.models.word2vec import Word2Vec, Text8Corpus
from gensim.models import KeyedVectors

logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)


def train(args):
    sentences = Text8Corpus(data_path)
    model = Word2Vec(sentences, size=args.dimension, window=args.window)
    model.save(args.save_file)


def calculate_similar_words(model):
    word = input('Input a word: ')
    pprint(model.most_similar(word))


def calculate_word_similarity(model):
    words_str = input('Input two words(e.g. 猫,犬): ')
    word1, word2 = words_str.split(',')
    print(model.similarity(word1, word2))


def predict(args):
    print('Loading model...')
    model = KeyedVectors.load(args.save_file)
    while True:
        print('m: Calculate similar words.')
        print('s: Calculate word similarity.')
        print('e: Exit.')
        choice = input('Your choice: ')
        if choice == 'm':
            calculate_similar_words(model)
        elif choice == 's':
            calculate_word_similarity(model)
        elif choice == 'e':
            break
        else:
            print('Please input m, s or e!')

In [ ]:
if __name__ == '__main__':
    parser = argparse.ArgumentParser(description='Training a word2vec model')
    parser.add_argument('--data_path', default='data/ja.text8', help='path to dataset')
    parser.add_argument('--save_file', default='model/model.bin', help='save file')
    parser.add_argument('--dimension', default=100, type=int, help='embedding dimension')
    parser.add_argument('--window', default=5, type=int, help='window size')
    parser.add_argument('--predict', action='store_true')
    args = parser.parse_args()
    if args.predict:
        predict(args)
    else:
        train(args)

In [5]:
dict_.keys()

dict_keys(['68', '86', '96', '40', '29', '78', '28', '95', '39', '12', '37', '32', '111', '110', '103', '1', '79', '93', '33', '118', '0', '89', '31', '9', '34', '21', '10', '8', '17', '46', '13', '61', '15', '23', '101', '69', '71', '64', '27', '26', '25', '67', '24', '114', '124', '74', '106', '3', '82', '109', '108', '126', '65', '91', '75', '123', '127', '59', '87', '88', '100', '43', '30', '121', '22', '36', '117', '92', '20', '113', '4', '11', '112', '56', '14', '45', '55', '6', '81', '48', '116', '104', '2', '73', '57', '18', '99', '115', '60', '84', '85', '5', '62', '49', '77', '76', '63', '119', '122', '120', '125', '44', '52', '35', '47', '50', '94', '128', '38', '19', '16', '107', '7', '54', '105', '58', '90', '80', '53', '98', '41', '72', '70', '66', '51', '102', '83', '97', '42'])

In [ ]:
replace_dict = {'[':'', ']':'', '/':'', '+':'', '(':'', ')':'', '等':'', ',':'', '.':'', '<':'', '>':'', '-':'', '?':'', ':':''}
exclude_task_list = ['new', 'rt', '籏智広太']
for key in dict_.keys():
    text_word_segmentation_list = []
    for text in dict_[key]['text']:
        text_word_segmentation_list.append(word_segmentation(text, replace_dict, exclude_task_list))
    print(text_word_segmentation_list)
    dict_[key]['text_word_segmentation'] = text_word_segmentation_list

['新型コロナウイルス の ワクチン接種 が 変異ウイルス を 生み出し た と する 言説 が 拡散 さ れ て い ます ノーベル賞科学者 の 発言 として 広がっ て い ます が これ は 誤り 変異 を 生み出さ ない ため に は むしろ ワクチン が 大切 です ', ' 新型コロナウイルス の ワクチン接種 が 変異ウイルス を 生み出し た と する 言説 が 拡散 さ れ て い ます ノーベル賞科学者 の 発言 として 広がっ て い ます が これ は 誤り 変異 を 生み出さ ない ため に は むしろ ワクチン が 大切 です ', ' 新型コロナウイルス の ワクチン接種 が 変異ウイルス を 生み出し た と する 言説 が 拡散 さ れ て い ます ノーベル賞科学者 の 発言 として 広がっ て い ます が これ は 誤り 変異 を 生み出さ ない ため に は むしろ ワクチン が 大切 です ', ' 新型コロナウイルス の ワクチン接種 が 変異ウイルス を 生み出し た と する 言説 が 拡散 さ れ て い ます ノーベル賞科学者 の 発言 として 広がっ て い ます が これ は 誤り 変異 を 生み出さ ない ため に は むしろ ワクチン が 大切 です ', ' 新型コロナウイルス の ワクチン接種 が 変異ウイルス を 生み出し た と する 言説 が 拡散 さ れ て い ます ノーベル賞科学者 の 発言 として 広がっ て い ます が これ は 誤り 変異 を 生み出さ ない ため に は むしろ ワクチン が 大切 です ', ' 新型コロナウイルス の ワクチン接種 が 変異ウイルス を 生み出し た と する 言説 が 拡散 さ れ て い ます ノーベル賞科学者 の 発言 として 広がっ て い ます が これ は 誤り 変異 を 生み出さ ない ため に は むしろ ワクチン が 大切 です ', ' 新型コロナウイルス の ワクチン接種 が 変異ウイルス を 生み出し た と する 言説 が 拡散 さ れ て い ます ノーベル賞科学者 の 発言 として 広がっ て い ます が これ は 誤り 変異 を 生み出さ ない ため に は むしろ ワクチン が 大切 です ', ' 新型コロ

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "/Users/kanekotakafumi/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py", line 3437, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-5-8e75245db5e1>", line 6, in <module>
    text_word_segmentation_list.append(word_segmentation(text, replace_dict, exclude_task_list))
  File "<ipython-input-3-218b200969f4>", line 5, in word_segmentation
    tokenizer = Tokenizer()
  File "/Users/kanekotakafumi/anaconda3/lib/python3.7/site-packages/janome/tokenizer.py", line 177, in __init__
  File "/Users/kanekotakafumi/anaconda3/lib/python3.7/site-packages/janome/sysdic/__init__.py", line 91, in mmap_entries
OSError: [Errno 24] Too many open files: '/Users/kanekotakafumi/anaconda3/lib/python3.7/site-packages/janome/sysdic/entries_extra8.py'

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/Users/kanekotakafumi/anaconda3/lib/python3.7/si

In [10]:
# analyzerモジュールで形態素分析
char_filters = [UnicodeNormalizeCharFilter()]

tokenizer = Tokenizer()

token_filters = [CompoundNounFilter(),
                 POSStopFilter(['記号','助詞']),
                 LowerCaseFilter()]

a = Analyzer(char_filters=char_filters, tokenizer=tokenizer ,token_filters=token_filters)

text_list = dict_['68']['text']

for text in text_list[:10]: # お試しで10行解析してみる
    for token in a.analyze(text):
        print(token)

new	名詞,固有名詞,組織,*,*,*,new,*,*
新型コロナウイルス	名詞,複合,*,*,*,*,新型コロナウイルス,シンガタコロナウイルス,シンガタコロナウイルス
ワクチン接種	名詞,複合,*,*,*,*,ワクチン接種,ワクチンセッシュ,ワクチンセッシュ
変異ウイルス	名詞,複合,*,*,*,*,変異ウイルス,ヘンイウイルス,ヘンイウイルス
生み出し	動詞,自立,*,*,五段・サ行,連用形,生み出す,ウミダシ,ウミダシ
た	助動詞,*,*,*,特殊・タ,基本形,た,タ,タ
する	動詞,自立,*,*,サ変・スル,基本形,する,スル,スル
言説	名詞,一般,*,*,*,*,言説,ゲンセツ,ゲンセツ
拡散	名詞,サ変接続,*,*,*,*,拡散,カクサン,カクサン
さ	動詞,自立,*,*,サ変・スル,未然レル接続,する,サ,サ
れ	動詞,接尾,*,*,一段,連用形,れる,レ,レ
い	動詞,非自立,*,*,一段,連用形,いる,イ,イ
ます	助動詞,*,*,*,特殊・マス,基本形,ます,マス,マス
ノーベル賞科学者	名詞,複合,*,*,*,*,ノーベル賞科学者,ノーベルショウカガクシャ,ノーベルショーカガクシャ
発言	名詞,サ変接続,*,*,*,*,発言,ハツゲン,ハツゲン
広がっ	動詞,自立,*,*,五段・ラ行,連用タ接続,広がる,ヒロガッ,ヒロガッ
い	動詞,非自立,*,*,一段,連用形,いる,イ,イ
ます	助動詞,*,*,*,特殊・マス,基本形,ます,マス,マス
これ	名詞,代名詞,一般,*,*,*,これ,コレ,コレ
誤り	名詞,一般,*,*,*,*,誤り,アヤマリ,アヤマリ
変異	名詞,サ変接続,*,*,*,*,変異,ヘンイ,ヘンイ
生み出さ	動詞,自立,*,*,五段・サ行,未然形,生み出す,ウミダサ,ウミダサ
ない	助動詞,*,*,*,特殊・ナイ,基本形,ない,ナイ,ナイ
ため	名詞,非自立,副詞可能,*,*,*,ため,タメ,タメ
むしろ	副詞,一般,*,*,*,*,むしろ,ムシロ,ムシロ
ワクチン	名詞,一般,*,*,*,*,ワクチン,ワクチン,ワクチン
大切	名詞,形容動詞語幹,*,*,*,*,大切,タイセツ,タイセツ
です	助動詞,*,*,*,特殊・デス,基本形,です,デス,デス
(籏智広太	名詞,複合,*,*,*,*,(籏智広太,**フトシ,**フト